In [1]:
import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,Sequential,load_model,Input

from keras.applications import MobileNet,MobileNetV2,InceptionV3
from keras.applications.mobilenet_v2 import preprocess_input
from keras.layers import AveragePooling2D,Flatten,Dense,Dropout,MaxPooling2D
from keras.optimizers import Adam

from keras.preprocessing.image import img_to_array,load_img

from sklearn.metrics import confusion_matrix,classification_report

In [2]:
datagen=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,
                             horizontal_flip=True,fill_mode="nearest")

train_data=datagen.flow_from_directory('archive/train/',target_size=(299,299),class_mode="categorical",batch_size=64)


test_data=datagen.flow_from_directory('archive/train/',target_size=(299,299),class_mode="categorical",batch_size=64)

Found 2536 images belonging to 2 classes.
Found 2536 images belonging to 2 classes.


In [3]:
pwd


'C:\\Users\\HP\\Desktop\\main_project'

In [4]:
train_data.class_indices

{'autistic': 0, 'non_autistic': 1}

In [5]:
bot_model=InceptionV3(weights='imagenet',include_top=False,input_tensor=Input(shape=(299,299,3)))

top_model=bot_model.output
top_model=MaxPooling2D(pool_size=(7,7))(top_model)
top_model=Flatten()(top_model)
top_model = Dense(128, activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(128, activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(128, activation="relu")(top_model)
top_model = Dropout(0.5)(top_model)
top_model = Dense(2, activation="softmax")(top_model)

In [6]:
fin_model = Model(inputs=bot_model.input, outputs=top_model)

In [7]:
for layer in bot_model.layers:
    layer.trainable = False

In [8]:
opti = Adam(lr=0.0001, decay=0.0001 /20)
fin_model.compile(loss="categorical_crossentropy", optimizer=opti,metrics=["accuracy"])

In [9]:
history = fin_model.fit_generator(train_data,steps_per_epoch=10,
                    validation_data=test_data,validation_steps=10,
                    epochs=20)


C:\Users\HP\Anaconda3\envs\projectmain\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
10/10 [==============================] - 92s 9s/step - loss: 47.0277 - accuracy: 0.5181 - val_loss: 9.9045 - val_accuracy: 0.5437
Epoch 2/20
10/10 [==============================] - 85s 9s/step - loss: 37.0751 - accuracy: 0.4734 - val_loss: 8.6477 - val_accuracy: 0.5109
Epoch 3/20
10/10 [==============================] - 94s 10s/step - loss: 19.8782 - accuracy: 0.5436 - val_loss: 3.2770 - val_accuracy: 0.5531
Epoch 4/20
10/10 [==============================] - 86s 9s/step - loss: 14.4670 - accuracy: 0.5163 - val_loss: 1.9185 - val_accuracy: 0.5281
Epoch 5/20
10/10 [==============================] - 87s 9s/step - loss: 9.9794 - accuracy: 0.5346 - val_loss: 1.3420 - val_accuracy: 0.5719
Epoch 6/20
10/10 [==============================] - 130s 14s/step - loss: 5.9773 - accuracy: 0.5087 - val_loss: 1.0571 - val_accuracy: 0.5188
Epoch 7/20
10/10 [==============================] - 86s 9s/step - loss: 3.4301 - accuracy: 0.5325 - val_loss: 0.9735 - val_accuracy: 0.4828
Epoch 8/20
10

In [ ]:
names=np.array(['autistic','non_autistic'])
image=load_img('archive/test/autistic/012.jpg',target_size=(299,299))
img=img_to_array(image)
ex_im=np.expand_dims(img,axis=0)
class2=fin_model.predict(ex_im)
names[np.argmax(class2)]

In [ ]:
top_model.save('autisim.h5')